# CV reviewer and rewrite

In [1]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import PromptTemplate
from langsmith import Client

In [2]:
client = Client()

In [3]:
model = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0.05,
    timeout=None,
    max_retries=2,
)

## Prompts for CV review and rewrite

In [4]:
template = """
You are an assistant for improving CVs for a given job description.
Use the following job description and CV to suggest improvements.
Try not to overuse words like "spearheaded", "expert", "innovative", but do not be afraid to suggest changes.
Quantify where possible, but do not make up any information, only use the information provided.
If you think key skills or informationare missing, suggest what they should be.
Keep the answer concise and in British English.

{job_description}

{cv}

Suggested improvements to the CV:
"""

review_prompt = PromptTemplate.from_template(template)
review_chain = review_prompt | model

try:
    client.push_prompt("cv-reviewer", object=review_chain)
except Exception as e:
    print("Prompt already exists, skipping push")


Prompt already exists, skipping push


In [5]:
# Step 2: Rewrite sections of the CV based on the suggestions
rewrite_template = """
Given the following CV and the suggested improvements, rewrite the relevant sections of the CV incorporating the suggestions.
Only rewrite the sections that need improvement based on the suggestions. Keep the rest of the CV unchanged.
If you are unsure if the person has the skill, leave it unchanged.
Do not make up information, only use the information provided.
If you are unsure of what to rewrite, leave it unchanged.

CV:
{cv}

Suggested improvements:
{suggestions}

Rewritten CV sections:
"""

rewrite_prompt = PromptTemplate.from_template(rewrite_template)
rewrite_chain = rewrite_prompt | model

try:
    client.push_prompt("cv-rewrite", object=rewrite_chain)
except Exception as e:
    print("Prompt already exists, skipping push")


Prompt already exists, skipping push


In [6]:
review_chain = client.pull_prompt("cv-reviewer", include_model=True)
rewrite_chain = client.pull_prompt("cv-rewrite", include_model=True)

In [7]:
review_chain

PromptTemplate(input_variables=['cv', 'job_description'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'cv-reviewer', 'lc_hub_commit_hash': 'e4204f07ff6a8fcacd8aa41f4c82ddbd156aac4aba733c3f1a5945b6a0f94279'}, template='\nYou are an assistant for improving CVs for a given job description.\nUse the following job description and CV to suggest improvements.\nTry not to overuse words like "spearheaded", "expert", "innovative", but do not be afraid to suggest changes.\nQuantify where possible, but do not make up any information, only use the information provided.\nIf you think key skills or informationare missing, suggest what they should be.\nKeep the answer concise and in British English.\n\n{job_description}\n\n{cv}\n\nSuggested improvements to the CV:\n')
| ChatAnthropic(model='claude-3-5-sonnet-20241022', temperature=0.05, anthropic_api_url='https://api.anthropic.com', anthropic_api_key=SecretStr('**********'), model_kwargs={})

In [8]:
def run(job_description: str, cv: str) -> tuple[str, str]:
    # Remove the tools and agent initialization since no tools are being used
    review_result = review_chain.invoke({
        "job_description": job_description,
        "cv": cv
    })
    suggestions = review_result.content

    print("Suggested improvements to the CV:")
    print(suggestions)

    rewrite_result = rewrite_chain.invoke({
        "cv": cv,
        "suggestions": suggestions
    })
    rewritten_sections = rewrite_result.content

    print("\nRewritten CV sections:")
    print(rewritten_sections)

    return suggestions, rewritten_sections

## Run full CV code

In [14]:
with open("tmp/cv.txt", "r") as file:
    cv = file.read()

In [24]:
with open("tmp/waymo.txt", "r") as file:
    job_description = file.read()

In [25]:
r = run(job_description, cv)

Suggested improvements to the CV:
Based on the Waymo job description, here are suggested improvements to the CV:

1. Experience Section Emphasis:
- Highlight simulation experience more prominently if any exists
- Add specific metrics around distributed systems scale and performance
- Emphasize any experience with agent-based modelling or synthetic data generation
- Include examples of productionizing research, as this is a key requirement

2. Skills Section Reorganization:
- Create a separate section for "Machine Learning & Simulation" skills
- Move GPU Optimisation and Distributed Systems to the forefront
- Add any experience with traffic/motion simulation software
- Include any computer vision or 3D modelling experience

3. Missing Key Elements:
- Experience with realistic environment simulation
- Foundation model development at scale
- Metrics development for model evaluation
- Team collaboration across multiple locations
- Experience with large datasets

4. Project Section Addition

## Rewrite section code

In [22]:
# Step 3: Rewrite specific section of the CV based on the suggestions
rewrite_section_template = """
Given the following CV section and the suggested improvements, rewrite the relevant sections of the CV incorporating the suggestions where appropriate.
If you are unsure if the person has the skill, leave it unchanged.
Do not make up information, only use the information provided.
If you are unsure of what to rewrite, leave it unchanged.

CV:
{cv_section}

Suggested improvements:
{suggestions}

Rewritten CV sections:
"""

rewrite_section_prompt = PromptTemplate.from_template(rewrite_section_template)
rewrite_section_chain = rewrite_section_prompt | model

try:
    client.push_prompt("cv-rewrite", object=rewrite_section_chain)
except Exception as e:
    print("Prompt already exists, skipping push")


In [23]:
def run_section(job_description: str, cv_section: str) -> tuple[str, str]:
    # Remove the tools and agent initialization since no tools are being used
    review_result = review_chain.invoke({
        "job_description": job_description,
        "cv": cv
    })
    suggestions = review_result.content

    print("Suggested improvements to the CV:")
    print(suggestions)

    rewrite_result = rewrite_section_chain.invoke({
        "cv_section": cv_section,
        "suggestions": suggestions
    })
    rewritten_section = rewrite_result.content

    print("\nRewritten CV sections:")
    print(rewritten_section)

    return suggestions, rewritten_section

## Run rewrite section code

In [36]:
cv_section = """
Machine Learning Engineer, Clarity AI - UK, Remote
November 2022 - November 2024
Architected and deployed generative AI solution using LLMs with LangChain, reducing false positives by 80% in news analysis systems. Implemented prompt engineering with continuous human-evaluation for iterative optimisation.
Engineered and optimised enterprise-scale NLP news analysis pipeline, using AWS, Airflow, and Spark, processing 300,000 daily articles, achieving 99% reliability and 85% inference time reduction.
"""

In [37]:
r = run_section(job_description, cv_section)

Suggested improvements to the CV:
Based on the Apple job description, here are the suggested improvements to the CV:

1. Skills Alignment:
- Add Swift and Xcode experience if you have any (these are key requirements)
- Highlight any experience with Apple's development APIs
- Emphasize code generation experience with LLMs more prominently
- Add specific examples of fine-tuning LLMs

2. Experience Section:
- Restructure the Clarity AI role to emphasize:
  * Any code generation aspects of the LLM work
  * Specific fine-tuning experiences
  * Cross-functional collaboration examples
  * Model evaluation metrics and deployment strategies

3. Missing Critical Elements:
- Experience with Apple's ecosystem (if any)
- Specific examples of code generation projects
- Direct experience with Swift programming
- Work with developer documentation or tools

4. Suggested Additions:
- If you have any open-source contributions related to ML/code generation
- Any experience developing developer tools
- Spe

## Cover letter code

In [21]:
# Step 3: Rewrite specific section of the CV based on the suggestions
write_cover_letter_template = """
Given the following CV section and the job description, write a cover letter for the job.
Do not make up information, only use the information provided.

CV:
{cv}

Job description:
{job_description}

Cover letter:
"""

write_cover_letter_prompt = PromptTemplate.from_template(write_cover_letter_template)
write_cover_letter_chain = write_cover_letter_prompt | model

try:
    client.push_prompt("write-cover-letter", object=write_cover_letter_chain)
except Exception as e:
    print("Prompt already exists, skipping push")


In [22]:
def run_cover_letter(job_description: str, cv: str) -> tuple[str, str]:
    # Remove the tools and agent initialization since no tools are being used
    cover_letter_result = write_cover_letter_chain.invoke({
        "job_description": job_description,
        "cv": cv
    })
    cover_letter = cover_letter_result.content

    print("Cover letter:")
    print(cover_letter)

    return cover_letter

In [23]:
r = run_cover_letter(job_description, cv)

Cover letter:
Dear Hiring Manager,

I am writing to express my strong interest in the Agent Engineering position at Hugging Face. With over three years of experience building production-scale AI solutions and a particular focus on Generative AI and LLMs, I believe my background aligns well with your mission to advance the field of LLM agents.

In my current role at Clarity AI, I've developed and optimized GenAI solutions using LangChain, implementing sophisticated prompt engineering techniques that resulted in an 80% reduction in false positives. This experience with orchestrating LLM interactions and building reliable AI systems directly relates to the agent development work you're pursuing. Additionally, my work on distributed NLP pipelines processing 300k+ daily articles demonstrates my ability to build and maintain large-scale AI systems with high reliability.

Your focus on GUI agents particularly excites me, as my experience with fine-tuning models and implementing HITL systems w